In [50]:
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.svm import LinearSVC
from collections import defaultdict
from tqdm import tqdm

In [51]:
def load_data(data_path):
    def sparse_to_dense(tf_idfs, vocab_size):
    #đọc DL
        r_d = [0.0 for _ in range(vocab_size)]
        index_tfidfs = tf_idfs.split()
        for index_tfidf in index_tfidfs:
            index = int(index_tfidf.split(":")[0])
            tfidf = float(index_tfidf.split(":")[1])
            r_d[index] = tfidf
        return r_d
    with open(data_path, 'r') as f:
        d_lines = f.read().splitlines()
    with open('./datasets/20news-bydate/words_idfs.txt', 'r') as f:
        vocab_size = len(f.read().splitlines())
    labels = []
    data = []
    for d in d_lines:
        features = d.split("<fff>")
        data.append(sparse_to_dense(tf_idfs=features[2], vocab_size=vocab_size))
        labels.append(int(features[0]))
    return data, labels

In [52]:
def clustering_with_Kmeans():
    data, labels = load_data (data_path = './datasets/20news-bydate/words_tf_idf.txt')
    #sử dụng csr_matrix để tạo ma trận thưa với các hàng được phân chia rõ rệt
    X= csr_matrix(data)
    from sklearn.cluster import KMeans
    kmean = KMeans(
        n_clusters=20,
        init='random',
        n_init=5, #số lần kmeans chạy với những lần khởi tạo centroid khác nhau
        tol= 1e-3, #ngưỡng tối thiểu cho độ giảm lỗi 
        random_state=2018
    ).fit(X)
    labels = kmean.labels_

In [53]:
def compute_accuracy(predicted_Y, expected_Y):
    matches = np.equal(predicted_Y, expected_Y)
    accuracy = np.sum(matches.astype(float) / len(expected_Y))
    return accuracy

In [54]:
def classifying_with_linear_SVMs():
    #dùng Sckit-learn cùng linear SVMs
    train_X, train_y = load_data(data_path= './datasets/20news-bydate/20news-train-tfidf.txt')
    classifier = LinearSVC(
        C= 10.0, #penalty coeff
        tol= 0.001, #Tolerance for stopping criteria.
        verbose=True #whether prints out logs or not
    )
    classifier.fit(train_X, train_y)
    test_X, test_y = load_data(data_path = './datasets/20news-bydate/20news-test-tfidf.txt')
    predicted_y = classifier.predict(test_X)
    accuracy = compute_accuracy(predicted_y = predicted_y, expected_y = test_y)
    print('Accuracy: ', accuracy)


In [ ]:
if __name__ == '__main__':
    clustering_with_Kmeans()
    classifying_with_linear_SVMs()